In [135]:
import pandas as pd 
import numpy as np
from datetime import datetime

def str_to_datetime(datelist,ymd_type):
    datelist = datelist.dropna()
    for i,dat in enumerate(datelist):
        datelist[i] = datetime.strptime(dat, ymd_type)
    return datelist

In [136]:
## display all the columns
pd.options.display.max_columns = 120
pd.options.display.max_rows = 120

In [233]:
df = pd.read_csv('movies_after2014_IMDB.csv')

In [7]:
#Creating the data for Yiran&Yuxin --- twitter date data 
#Select +-3 days time window
df.release_date = str_to_datetime(df.release_date,'%Y-%m-%d %H:%M:%S')

import datetime
df['start_date'] = df.release_date - datetime.timedelta(days = 3)
df['end_date'] = df.release_date + datetime.timedelta(days = 3)

df.release_date

df2 = df[['original_title','start_date','end_date']]

df2.to_csv('movielist_S2E.csv')

df3 = pd.read_csv('movielist_S2E.csv')
df3.head()

In [21]:
#Reading the dataset from IMDB website --- many dataset
name_basics = pd.read_csv('imdb/name_basics.tsv',sep='\t')
title_ratings = pd.read_csv('imdb/title_ratings.tsv',sep='\t')
title_episode = pd.read_csv('imdb/title_episode.tsv',sep='\t')
title_crews = pd.read_csv('imdb/title_crew.tsv',sep='\t')
title_basics = pd.read_csv('imdb/title_basics.tsv',sep='\t')
title_akas = pd.read_csv('imdb/title_akas.tsv',sep='\t')

print(name_basics.columns)
print(title_ratings.columns)
print(title_episode.columns)
print(title_crews.columns)
print(title_basics.columns)
print(title_akas.columns)

C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
df3 = pd.merge(title_basics,df,'right',left_on='primaryTitle',right_on='title')
df4 = df3[df3['titleType']=='movie']
len(df3['primaryTitle'].unique())

541

In [234]:
#Merge the dataset after 2014 with the metadata dataset --- right join containing Nan
metadata = pd.read_csv('movie_metadata.csv')

for i,mov in enumerate(metadata.movie_title):
    metadata.movie_title[i] = mov.replace(u'\xa0', u'')

merged_data = pd.merge(metadata,df,'right',left_on='movie_title',right_on='title')

merged_data = pd.read_csv('merged_IMDB_and_metadata.csv')
merged_data

temp = merged_data.title.value_counts() > 1
duplicate_list = temp[temp == True].index


for i,name in enumerate(duplicate_list):
    temp = merged_data[merged_data['title']== name].iloc[-1:]
    merged_data = merged_data[merged_data['title'] != name]
    merged_data = pd.concat([merged_data,temp])



merged_data = merged_data.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'])

merged_data.to_csv('merged_IMDB_and_metadata.csv',index=False)

C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [253]:
##Delete missing value
merged_data_noNan = merged_data[merged_data['movie_title'].isnull().values == False]
merged_data_noNan.to_csv('merged_data_innerJoin.csv',index=False)